## <div style="text-align: center"> TFM - Parte 2 </div>
<div style="text-align: left"><b>Autor</b>: Santiago Alonso Gutiérrez </div>
<div style="text-align: left"><b>Propuesta elegida</b>: 2 (Préstamos) </div>
<div style="text-align: left"><b>Conjunto de datos</b>: OpenML/credit-g </div>
<div style="text-align: left"><b>URL</b>: https://www.openml.org/d/31 </div>
<div style="text-align: left"><b>Descripción</b>: Base de datos de clasificación de personas según su riesgo para conceder un préstamo. Tiene 1000 instancias y 20 variables descriptivas que son indicadores del estado económico de la persona. La mayoría de los atributos son nominales. La última variable, la vigésimo primera, indica la clase (good o bad). El error cometido al clasificar un cliente malo como bueno es más grave que el cometido al clasificar uno bueno como malo.</div>
<div style="text-align: left"><b>Modelos a usar</b>: SVM, MLP y Boosting</div>

# Importamos los paquetes necesarios

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pprint import pprint

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder  
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA

import sklearn.preprocessing
from sklearn.svm import SVC
import scipy.stats as stats

# Desactivamos los warnings 

In [2]:
import warnings
warnings.filterwarnings("ignore")

# Cargando el conjunto de datos

Empezamos definiendo los path con los que vamos a trabajar:

In [3]:
ROOT_DIR = os.path.abspath("")
DATA_DIR = os.path.join(ROOT_DIR, "data")

Con esto, cargamos el conjunto de datos accediendo a la URL de la base de datos.

In [4]:
# Tras acceder a https://www.openml.org/d/31 desde Google Chrome, pulsamos F12 e inspeccionamos el elemento del icono CSV,
# obteniendo la URL de descarga directa del fichero, la cual guardaremos en la constante CREDITBASE_URL para su posterior uso:
CREDITBASE_URL = "https://www.openml.org/data/get_csv/31/dataset_31_credit-g.arff"

# Le pasamos la constante CREDITBASE_URL a la función read_csv de Pandas, la cual leerá y descargará el conjunto de datos,
# devolviendonos un objeto de tipo DataFrame que llamaremos df_credit:
df_credit = pd.read_csv(CREDITBASE_URL)

df_credit.head()

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,...,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker,class
0,'<0',6,'critical/other existing credit',radio/tv,1169,'no known savings','>=7',4,'male single',none,...,'real estate',67,none,own,2,skilled,1,yes,yes,good
1,'0<=X<200',48,'existing paid',radio/tv,5951,'<100','1<=X<4',2,'female div/dep/mar',none,...,'real estate',22,none,own,1,skilled,1,none,yes,bad
2,'no checking',12,'critical/other existing credit',education,2096,'<100','4<=X<7',2,'male single',none,...,'real estate',49,none,own,1,'unskilled resident',2,none,yes,good
3,'<0',42,'existing paid',furniture/equipment,7882,'<100','4<=X<7',2,'male single',guarantor,...,'life insurance',45,none,'for free',1,skilled,2,none,yes,good
4,'<0',24,'delayed previously','new car',4870,'<100','1<=X<4',3,'male single',none,...,'no known property',53,none,'for free',2,skilled,2,none,yes,bad


# Definimos la semilla aleatoria

In [5]:
RANDOM_SEED = 123456789
np.random.seed(RANDOM_SEED)

# Descripción de las variables

|Nombre|Descripción|
|---|:---:|
| checking_status | Estado de la cuenta corriente existente, en marcos alemanes. | 
| duration | Duración en meses. | 
| credit_history | Historial crediticio (créditos tomados, devueltos debidamente, retrasos, cuentas críticas). | 
| purpose | Objeto del crédito (coche, televisión, ...). | 
| credit_amount | Monto del crédito. | 
| savings_status | Estado de las cuentas de ahorro / bonos, en marcos alemanes. | 
| employment | Empleo actual, en número de años. | 
| installment_commitment | Tasa de pago a plazos en porcentaje de la renta disponible. | 
| personal_status | Estado personal (casado, soltero, ...) y sexo. | 
| other_parties | Otros deudores / garantes. | 
| residence_since | Residencia actual desde hace X años. | 
| property_magnitude | Propiedad (por ejemplo, bienes raíces). | 
| age | Edad en años. | 
| other_payment_plans | Otros planes de pago (bancos, tiendas). | 
| housing | Vivienda (alquiler, propia, ...). | 
| existing_credits | Número de créditos existentes en este banco. | 
| job | Trabajo. | 
| num_dependents | Número de personas a las que están obligadas a mantener. | 
| own_telephone | Teléfono (sí, no). | 
| foreign_worker | Trabajador extranjero (sí, no). | 
| class | Este conjunto de datos clasifica a las personas descritas por un conjunto de atributos como riesgos crediticios buenos (good) o malos (bad). Es peor clasificar a un cliente como bueno cuando es malo (5), que clasificar a un cliente como malo cuando es bueno (1). | 

Describimos brevemente el `dataframe` con el que estamos trabajando:

In [6]:
df_credit.describe()

,duration,credit_amount,installment_commitment,residence_since,age,existing_credits,num_dependents
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,20.903000,3271.258000,2.973000,2.845000,35.546000,1.407000,1.155000
std,12.058814,2822.736876,1.118715,1.103718,11.375469,0.577654,0.362086
min,4.000000,250.000000,1.000000,1.000000,19.000000,1.000000,1.000000
25%,12.000000,1365.500000,2.000000,2.000000,27.000000,1.000000,1.000000
50%,18.000000,2319.500000,3.000000,3.000000,33.000000,1.000000,1.000000
75%,24.000000,3972.250000,4.000000,4.000000,42.000000,2.000000,1.000000
max,72.000000,18424.000000,4.000000,4.000000,75.000000,4.000000,2.000000


Comprobamos los tipos de cada variable:

In [7]:
obj = df_credit.dtypes

for key,value in obj.iteritems():
        print(key," -->",value)
        
        
df_credit["class"]

checking_status  --> object
duration  --> int64
credit_history  --> object
purpose  --> object
credit_amount  --> int64
savings_status  --> object
employment  --> object
installment_commitment  --> int64
personal_status  --> object
other_parties  --> object
residence_since  --> int64
property_magnitude  --> object
age  --> int64
other_payment_plans  --> object
housing  --> object
existing_credits  --> int64
job  --> object
num_dependents  --> int64
own_telephone  --> object
foreign_worker  --> object
class  --> object


0      good
1       bad
2      good
3      good
4       bad
       ... 
995    good
996    good
997    good
998     bad
999    good
Name: class, Length: 1000, dtype: object

Las variables tipo `object` son variables categóricas tipo *string*. Esto es problemático con los modelos *SVM* y *MLP*, que trabajan de forma óptima con datos numéricos.

# Procesado de datos

## Numerizamos la variable de salida

Empezamos haciendo el encoding de la variable `class` de cara a poder hacer modelos de clasificación más adelante. Es decir, pasamos de una variable tipo *string* a una variable tipo *int*:

In [8]:
df_credit['class'] =  df_credit['class'].map({
    'good': 0, 
     'bad': 1,
}).astype(int)

## Separamos el dataframe en train y test

Separamos los datos en *train* y *test*. Hacemos la división estratificada, es decir, teniendo en cuenta la clase de salida (para que la distribución de la variable de salida no se vea afectada en alguno de los dos subconjuntos generados):

In [9]:
df_train, df_test = train_test_split(df_credit, test_size=0.2, shuffle = True, stratify = df_credit['class'])

print("Tamaños de los dos dataframes generados")
print(df_train.shape)
print(df_test.shape)

Tamaños de los dos dataframes generados
(800, 21)
(200, 21)


## Pasamos todas las variables categóricas a numéricas

Los algoritmos que vamos a usar esperan trabajar con variables numéricas. Tenemos dos opciones:
1. Quedarnos solo con aquellas variables numéricas. Sin embargo, con esto estamos perdiendo demasiada información valiosa sobre el problema
2. Numerizar las variables categóricas. Esto puede introducir ciertos problemas
    1. Si se usa un entero para numerizar ("x": 1, "y": 2, "z": 3, ...) estamos introduciendo información que no estaba en el dataset (por ejemplo, "x" es la mitad que "z", o "x" está más cerca de "y" que "z")
    2. Si usamos *one-hot-encoding*, aumentamos drásticamente la dimensionalidad del conjunto de datos
    
Por simplicidad, decidimos numerizar las variables usando un entero, conscientes de los problemas que esto puede suponer. Además, la transformación se aprende sobre el conjunto de entrenamiento, y se aplica en training y test. Con esto, evitamos hacer *data snooping* sobre el conjunto de test.

In [10]:
# Usamos un label encoder de scikit learn para hacer el encoding

for var_name, var_type in  df_credit.dtypes.iteritems():
    if var_type == "object":
        
        # Calculamos el transformador sobre el conjunto de entrenamiento
        curr_encoder = LabelEncoder()
        df_train[var_name] = curr_encoder.fit_transform(df_train[var_name])
        
        # Aplicamos la transformacion aprendida sobre el conjunto de test
        # Con esto, evitamos hacer data snooping sobre el conjunto de test
        df_test[var_name] = curr_encoder.transform(df_test[var_name])

print("Resultado en training")
print(df_train.head())
print("")

print("Resultado en test")
print(df_test.head())

Resultado en training
     checking_status  duration  credit_history  purpose  credit_amount  \
846                3        18               3        1           6761   
335                1         6               1        5           3384   
946                1        24               0        5           3349   
347                0        24               3        7           3758   
81                 3        15               3        7           1213   

     savings_status  employment  installment_commitment  personal_status  \
846               4           0                       2                3   
335               2           0                       1                1   
946               1           2                       4                3   
347               1           4                       1                0   
81                1           3                       4                3   

     other_parties  ...  property_magnitude  age  other_payment_plans  \
846

## Borramos las filas con `NaN` o `None`

In [11]:
df_train = df_train.dropna()
df_train.shape

(800, 21)

Podemos ver que no hemos eliminado ningún valor.

## Borrado de outliers

Usamos el criterio $3 \cdot IQR$, variable por variable, como método para borrar los *outliers*. Es importante que esta fase la realicemos previamente a la normalización de los datos, para que los outliers no dominen las estadísticas que se usan en esta normalización.

In [12]:
# Guardamos el shape anterior para poder calcular cuantos datos hemos borrado
old_shape = df_train.shape

# Calculamos el zscore (numero de desviaciones tipicas que nos desviamos de la media)
z_score = np.abs(stats.zscore(df_train))

# Usamos esa metrica para filtrar los outliers
df_train = df_train[(z_score<3).all(axis=1)]

# Calculamos el numero de filas que hemos borrado
new_shape = df_train.shape
deleted_rows = old_shape[0] - new_shape[0]
print(f"Hemos eliminado {deleted_rows} filas")

Hemos eliminado 93 filas


## Separamos los datasets

In [13]:
X_train = df_train.iloc[:, :-1]
Y_train = df_train.iloc[:, -1]

X_test = df_test.iloc[:, :-1]
Y_test = df_test.iloc[:, -1]

print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(707, 20)
(707,)
(200, 20)
(200,)


## Normalizamos el conjunto de datos

Usamos min-max para normalizar. De nuevo, aprendemos los parámetros en train y aplicamos en train y test, para evitar *data snooping* en el conjunto de test.

In [14]:
scaler = sklearn.preprocessing.MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Selección de hiperparámetros usando K-Fold Cross Validation

Los algoritmos con los que vamos a trabajar son *SVM*, *MLP* y *AdaBoost*. Los hiperparámetros que definen estos modelos y su aprendizaje los escogemos usando *K-Fold Cross Validation*. Gracias a esta técnica podemos usar el *training set* tanto para entrenar con distintos parámetros como para validar dichos modelos. Se puede consultar más información sobre esta técnica en la [documentación oficial de sklearn](https://scikit-learn.org/stable/modules/cross_validation.html)

## Support Vector Machine

Exploramos todas las combinaciones de parámetros y aplicamos Cross Validation:

In [15]:
# Valores que fijan la penalizacion de regularizacion
C_values = [0.3, 0.5, 0.8, 0.9, 1.0, 1.05, 1.1, 1.15, 1.2, 1.3, 1.4, 1.5, 2.0, 3.0, 5.0]

# Tipos de kernel que podemos aplicar
kernel_values = ["rbf", "linear", "poly"]

# Diccionario en el que vamos a guardar los resultados de CV
results = dict()

# Para hacer 5 fold cross validation
cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=RANDOM_SEED)

# Para precomputar la mejor combinacion de parametros
best_parameters = {
    "C": None,
    "kernel": None,
}
best_acc = 0.0

# Probamos con todas las combinaciones de parametros
for C in C_values:
    for kernel in kernel_values:
        
        # Para ver el proceso de ejecucion
        print(f"Computando CV para {C}, {kernel}")
        
        # Definimos el clasificador, usando los parametros que movemos y fijando
        # el resto de parametros necesarios. 
        clf = SVC(kernel=kernel, C=C, random_state=RANDOM_SEED)
        
        # Aplicamos el algoritmo 
        scores = cross_val_score(clf, X_train, Y_train, cv=cv, scoring='accuracy')
        
        # Guardamos los resultados
        results[C, kernel, "mean"] = scores.mean()
        results[C, kernel, "std"] = scores.std()
        
        # Comprobamos si hemos mejorado los resultados
        if results[C, kernel, "mean"] > best_acc:
            best_acc = results[C, kernel, "mean"]
            best_parameters = {
                "C": C,
                "kernel": kernel
            }

Computando CV para 0.3, rbf
Computando CV para 0.3, linear
Computando CV para 0.3, poly
Computando CV para 0.5, rbf
Computando CV para 0.5, linear
Computando CV para 0.5, poly
Computando CV para 0.8, rbf
Computando CV para 0.8, linear
Computando CV para 0.8, poly
Computando CV para 0.9, rbf
Computando CV para 0.9, linear
Computando CV para 0.9, poly
Computando CV para 1.0, rbf
Computando CV para 1.0, linear
Computando CV para 1.0, poly
Computando CV para 1.05, rbf
Computando CV para 1.05, linear
Computando CV para 1.05, poly
Computando CV para 1.1, rbf
Computando CV para 1.1, linear
Computando CV para 1.1, poly
Computando CV para 1.15, rbf
Computando CV para 1.15, linear
Computando CV para 1.15, poly
Computando CV para 1.2, rbf
Computando CV para 1.2, linear
Computando CV para 1.2, poly
Computando CV para 1.3, rbf
Computando CV para 1.3, linear
Computando CV para 1.3, poly
Computando CV para 1.4, rbf
Computando CV para 1.4, linear
Computando CV para 1.4, poly
Computando CV para 1.5, rb

Mostramos todos los parámetros con sus métricas alcanzadas:

In [16]:
pprint(results)

{(0.3, 'linear', 'mean'): 0.7140845070422536,
 (0.3, 'linear', 'std'): 0.036347853241025185,
 (0.3, 'poly', 'mean'): 0.7169014084507042,
 (0.3, 'poly', 'std'): 0.023314007545418108,
 (0.3, 'rbf', 'mean'): 0.7140845070422536,
 (0.3, 'rbf', 'std'): 0.036619718309859155,
 (0.5, 'linear', 'mean'): 0.7105633802816902,
 (0.5, 'linear', 'std'): 0.034420702562957574,
 (0.5, 'poly', 'mean'): 0.7126760563380281,
 (0.5, 'poly', 'std'): 0.02222497723670354,
 (0.5, 'rbf', 'mean'): 0.717605633802817,
 (0.5, 'rbf', 'std'): 0.03811284261333749,
 (0.8, 'linear', 'mean'): 0.7070422535211268,
 (0.8, 'linear', 'std'): 0.03509841068838623,
 (0.8, 'poly', 'mean'): 0.7084507042253522,
 (0.8, 'poly', 'std'): 0.024838298716265365,
 (0.8, 'rbf', 'mean'): 0.7197183098591549,
 (0.8, 'rbf', 'std'): 0.0235257649133663,
 (0.9, 'linear', 'mean'): 0.7084507042253521,
 (0.9, 'linear', 'std'): 0.036619718309859155,
 (0.9, 'poly', 'mean'): 0.7098591549295774,
 (0.9, 'poly', 'std'): 0.02288461522432667,
 (0.9, 'rbf', 'mea

Tomamos ahora la mejor combinación de parámetros:

In [17]:
print(best_parameters)
print(best_acc)

{'C': 1.1, 'kernel': 'rbf'}
0.730281690140845


## MLP

Exploramos todas las combinaciones de parámetros y aplicamos Cross Validation:

In [18]:
# Valores para la regularizacion
alpha_values = [0.001, 0.01, 0.1, 0.5]

# Valores del learning rate
lr_values = [0.001, 0.01, 0.1, 0.25, 0.5]

# Diccionario en el que vamos a guardar los resultados de CV
results = dict()

# Para hacer 5 fold cross validation
cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=RANDOM_SEED)

# Para precomputar la mejor combinacion de parametros
best_parameters = {
    "lr": None,
    "alpha": None,
}
best_acc = 0.0

# Probamos con todas las combinaciones de parametros
for lr in lr_values:
    for alpha in alpha_values:
        
        # Para ver el proceso de ejecucion
        print(f"Computando CV para {lr}, {alpha}")
        
        # Definimos el clasificador, usando los parametros que movemos y fijando
        # el resto de parametros necesarios
        clf = MLPClassifier(
            activation = "relu",
            solver = "adam",
            learning_rate_init = lr,
            alpha = alpha,
            max_iter = 10,
            random_state = RANDOM_SEED
        )
        
        # Aplicamos el algoritmo 
        scores = cross_val_score(clf, X_train, Y_train, cv=cv, scoring='accuracy')
        
        # Guardamos los resultados
        results[lr, alpha, "mean"] = scores.mean()
        results[lr, alpha, "std"] = scores.std()
        
        # Comprobamos si hemos mejorado los resultados
        if results[lr, alpha, "mean"] > best_acc:
            best_acc = results[lr, alpha, "mean"]
            best_parameters = {
                "lr": lr,
                "alpha": alpha
            }

Computando CV para 0.001, 0.001
Computando CV para 0.001, 0.01
Computando CV para 0.001, 0.1
Computando CV para 0.001, 0.5
Computando CV para 0.01, 0.001
Computando CV para 0.01, 0.01
Computando CV para 0.01, 0.1
Computando CV para 0.01, 0.5
Computando CV para 0.1, 0.001
Computando CV para 0.1, 0.01
Computando CV para 0.1, 0.1
Computando CV para 0.1, 0.5
Computando CV para 0.25, 0.001
Computando CV para 0.25, 0.01
Computando CV para 0.25, 0.1
Computando CV para 0.25, 0.5
Computando CV para 0.5, 0.001
Computando CV para 0.5, 0.01
Computando CV para 0.5, 0.1
Computando CV para 0.5, 0.5


Mostramos todos los parámetros con sus métricas alcanzadas:

In [19]:
pprint(results)

{(0.001, 0.001, 'mean'): 0.7133802816901408,
 (0.001, 0.001, 'std'): 0.03591549295774648,
 (0.001, 0.01, 'mean'): 0.7133802816901408,
 (0.001, 0.01, 'std'): 0.03591549295774648,
 (0.001, 0.1, 'mean'): 0.7133802816901408,
 (0.001, 0.1, 'std'): 0.03591549295774648,
 (0.001, 0.5, 'mean'): 0.7133802816901408,
 (0.001, 0.5, 'std'): 0.03591549295774648,
 (0.01, 0.001, 'mean'): 0.7042253521126761,
 (0.01, 0.001, 'std'): 0.029711300154547874,
 (0.01, 0.01, 'mean'): 0.7063380281690141,
 (0.01, 0.01, 'std'): 0.028353270618587886,
 (0.01, 0.1, 'mean'): 0.7042253521126761,
 (0.01, 0.1, 'std'): 0.028169014084507046,
 (0.01, 0.5, 'mean'): 0.7056338028169014,
 (0.01, 0.5, 'std'): 0.028133780782307303,
 (0.1, 0.001, 'mean'): 0.6992957746478874,
 (0.1, 0.001, 'std'): 0.03134396209051815,
 (0.1, 0.01, 'mean'): 0.7161971830985916,
 (0.1, 0.01, 'std'): 0.03421839609367984,
 (0.1, 0.1, 'mean'): 0.7140845070422535,
 (0.1, 0.1, 'std'): 0.03509841068838623,
 (0.1, 0.5, 'mean'): 0.7190140845070422,
 (0.1, 0.5,

Tomamos ahora la mejor combinación de parámetros:

In [20]:
print(best_parameters)
print(best_acc)

{'lr': 0.25, 'alpha': 0.1}
0.7204225352112676


## AdaBoost

Exploramos todas las combinaciones de parámetros y aplicamos Cross Validation:

In [21]:
# Numero de estimadores usados en el ensemble
n_values = [40, 50, 60, 75, 100, 125, 150, 175, 200, 400]

# Valores para el learning rate
lr_values = [0.4, 0.5, 0.6, 0.75, 1.0, 1.25, 1.5, 2.0]

# Diccionario en el que vamos a guardar los resultados de CV
results = dict()

# Para hacer 5 fold cross validation
cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=RANDOM_SEED)

# Para precomputar la mejor combinacion de parametros
best_parameters = {
    "lr": None,
    "n": None,
}
best_acc = 0.0

# Probamos con todas las combinaciones de parametros
for n in n_values:
    for lr in lr_values:
        
        # Para ver el proceso de ejecucion
        print(f"Computando CV para {lr}, {alpha}")
        
        # Definimos el clasificador, usando los parametros que movemos y fijando
        # el resto de parametros necesarios
        clf = AdaBoostClassifier(
            n_estimators = n,
            learning_rate = lr,
            random_state = RANDOM_SEED
        )
        
        # Aplicamos el algoritmo 
        scores = cross_val_score(clf, X_train, Y_train, cv=cv, scoring='accuracy')
        
        # Guardamos los resultados
        results[lr, n, "mean"] = scores.mean()
        results[lr, n, "std"] = scores.std()
        
        # Comprobamos si hemos mejorado los resultados
        if results[lr, n, "mean"] > best_acc:
            best_acc = results[lr, n, "mean"]
            best_parameters = {
                "lr": lr,
                "n": n
            }

Computando CV para 0.4, 0.5
Computando CV para 0.5, 0.5
Computando CV para 0.6, 0.5
Computando CV para 0.75, 0.5
Computando CV para 1.0, 0.5
Computando CV para 1.25, 0.5
Computando CV para 1.5, 0.5
Computando CV para 2.0, 0.5
Computando CV para 0.4, 0.5
Computando CV para 0.5, 0.5
Computando CV para 0.6, 0.5
Computando CV para 0.75, 0.5
Computando CV para 1.0, 0.5
Computando CV para 1.25, 0.5
Computando CV para 1.5, 0.5
Computando CV para 2.0, 0.5
Computando CV para 0.4, 0.5
Computando CV para 0.5, 0.5
Computando CV para 0.6, 0.5
Computando CV para 0.75, 0.5
Computando CV para 1.0, 0.5
Computando CV para 1.25, 0.5
Computando CV para 1.5, 0.5
Computando CV para 2.0, 0.5
Computando CV para 0.4, 0.5
Computando CV para 0.5, 0.5
Computando CV para 0.6, 0.5
Computando CV para 0.75, 0.5
Computando CV para 1.0, 0.5
Computando CV para 1.25, 0.5
Computando CV para 1.5, 0.5
Computando CV para 2.0, 0.5
Computando CV para 0.4, 0.5
Computando CV para 0.5, 0.5
Computando CV para 0.6, 0.5
Computando C

Mostramos todos los parámetros con sus métricas alcanzadas:

In [22]:
pprint(results)

{(0.4, 40, 'mean'): 0.7394366197183099,
 (0.4, 40, 'std'): 0.04069910782197151,
 (0.4, 50, 'mean'): 0.7380281690140846,
 (0.4, 50, 'std'): 0.040060458178388415,
 (0.4, 60, 'mean'): 0.7330985915492959,
 (0.4, 60, 'std'): 0.03555466355313899,
 (0.4, 75, 'mean'): 0.7253521126760563,
 (0.4, 75, 'std'): 0.03792369582489089,
 (0.4, 100, 'mean'): 0.7274647887323943,
 (0.4, 100, 'std'): 0.04260999861241085,
 (0.4, 125, 'mean'): 0.7232394366197183,
 (0.4, 125, 'std'): 0.039342190770319516,
 (0.4, 150, 'mean'): 0.728169014084507,
 (0.4, 150, 'std'): 0.038210309764086035,
 (0.4, 175, 'mean'): 0.7323943661971831,
 (0.4, 175, 'std'): 0.03895582165053334,
 (0.4, 200, 'mean'): 0.7267605633802817,
 (0.4, 200, 'std'): 0.03723747624338673,
 (0.4, 400, 'mean'): 0.7098591549295774,
 (0.4, 400, 'std'): 0.04615766086809559,
 (0.5, 40, 'mean'): 0.7380281690140845,
 (0.5, 40, 'std'): 0.0365655070021047,
 (0.5, 50, 'mean'): 0.7281690140845072,
 (0.5, 50, 'std'): 0.03923489655544823,
 (0.5, 60, 'mean'): 0.73098

Tomamos ahora la mejor combinación de parámetros:

In [23]:
print(best_parameters)
print(best_acc)

{'lr': 0.4, 'n': 40}
0.7394366197183099


# Entrenamiento usando los mejores parámetros

## Función para evaluar las clasificaciones

In [24]:
def display_classification_metrics(X_train, Y_train, X_test, Y_test, model = clf, outputs_prob = False):
    """Muestra algunas metricas en clasificacion"""
    metrics = dict()
    
    # Realizamos las predicciones
    predicted_training = clf.predict(X_train)
    predicted_test = clf.predict(X_test)
    
    # Accuracy obtenido
    metrics["train_acc"] = clf.score(X_train, Y_train)
    metrics["test_acc"]= clf.score(X_test, Y_test)
    
    # Calculamos el valor de F1
    metrics["train_f1"] = f1_score(Y_train, predicted_training, average='macro')
    metrics["test_f1"] = f1_score(Y_test, predicted_test, average='macro')

    # Calculamos el area bajo la curva ROC
    if outputs_prob is False:
        metrics["train_roc_auc"] = roc_auc_score(Y_train, clf.decision_function(X_train))
        metrics["test_roc_auc"] = roc_auc_score(Y_test, clf.decision_function(X_test))
    else:
        metrics["train_roc_auc"] = roc_auc_score(Y_train, clf.predict_proba(X_train)[:, 1])
        metrics["test_roc_auc"] = roc_auc_score(Y_test, clf.predict_proba(X_test)[:, 1])
        
    # Mostramos las metricas
    print("Metricas obtenidas:")
    pprint(metrics)
    print("")
    
    # Mostramos las matrices de confusion
    print("Matriz de confusion en training:")
    matrix_train = confusion_matrix(Y_train, predicted_training)
    pprint(matrix_train)
    print("")

    print("Matriz de confusion en test:")
    matrix_test = confusion_matrix(Y_test, predicted_test)
    pprint(matrix_test)
    print("")

Una vez que, con *Cross Validation*, hemos explorado los posibles parámetros, procedemos a entrenar los modelos con los mejores parámetros encontrados usando ese procedimiento:

## Support Vector Machine

In [25]:
# Definimos el modelo
kernel = "rbf"
C = 1.1
clf = SVC(kernel=kernel, C=C, random_state=RANDOM_SEED)

# Entrenamos con el training
clf.fit(X_train, Y_train)

# Evaluamos en ambos datasets
Y_pred_training = clf.predict(X_train)
Y_pred_test = clf.predict(X_test)

print(f"Accuracy en training: {clf.score(X_train, Y_train)}")
print(f"Accuracy en test: {clf.score(X_test, Y_test)}")

Accuracy en training: 0.7807637906647807
Accuracy en test: 0.73


Mostramos ahora métricas más avanzadas:

In [26]:
display_classification_metrics(X_train, Y_train, X_test, Y_test, model = clf)

Metricas obtenidas:
{'test_acc': 0.73,
 'test_f1': 0.593128390596745,
 'test_roc_auc': 0.7689285714285715,
 'train_acc': 0.7807637906647807,
 'train_f1': 0.653940283896231,
 'train_roc_auc': 0.8929565217391305}

Matriz de confusion en training:
array([[490,  10],
       [145,  62]], dtype=int64)

Matriz de confusion en test:
array([[131,   9],
       [ 45,  15]], dtype=int64)



## MLP

In [27]:
# Definimos el modelo
clf = MLPClassifier(
    activation = "relu",
    solver = "adam",
    learning_rate_init = 0.25,
    alpha = 0.1,
    max_iter = 10,
    random_state = RANDOM_SEED
)

# Entrenamos con el training
clf.fit(X_train, Y_train)

# Evaluamos en ambos datasets
Y_pred_training = clf.predict(X_train)
Y_pred_test = clf.predict(X_test)

print(f"Accuracy en training: {clf.score(X_train, Y_train)}")
print(f"Accuracy en test: {clf.score(X_test, Y_test)}")

Accuracy en training: 0.7072135785007072
Accuracy en test: 0.7


Mostramos ahora métricas más avanzadas:

In [28]:
display_classification_metrics(X_train, Y_train, X_test, Y_test, model = clf, outputs_prob = True)

Metricas obtenidas:
{'test_acc': 0.7,
 'test_f1': 0.4117647058823529,
 'test_roc_auc': 0.7591666666666667,
 'train_acc': 0.7072135785007072,
 'train_f1': 0.4142502071251036,
 'train_roc_auc': 0.6180966183574879}

Matriz de confusion en training:
array([[500,   0],
       [207,   0]], dtype=int64)

Matriz de confusion en test:
array([[140,   0],
       [ 60,   0]], dtype=int64)



## AdaBoost

In [29]:
# Definimos el modelo
clf = AdaBoostClassifier(
    n_estimators = 40,
    learning_rate = 0.4,
    random_state = RANDOM_SEED
)

# Entrenamos con el training
clf.fit(X_train, Y_train)

# Evaluamos en ambos datasets
Y_pred_training = clf.predict(X_train)
Y_pred_test = clf.predict(X_test)

print(f"Accuracy en training: {clf.score(X_train, Y_train)}")
print(f"Accuracy en test: {clf.score(X_test, Y_test)}")

Accuracy en training: 0.7751060820367751
Accuracy en test: 0.765


Mostramos ahora métricas más avanzadas:

In [30]:
display_classification_metrics(X_train, Y_train, X_test, Y_test, model = clf)

Metricas obtenidas:
{'test_acc': 0.765,
 'test_f1': 0.6489262371615313,
 'test_roc_auc': 0.8282738095238095,
 'train_acc': 0.7751060820367751,
 'train_f1': 0.6676580585078863,
 'train_roc_auc': 0.8134251207729469}

Matriz de confusion en training:
array([[475,  25],
       [134,  73]], dtype=int64)

Matriz de confusion en test:
array([[134,   6],
       [ 41,  19]], dtype=int64)



# Repetimos todo el proceso, pero usando PCA

Usamos la técnica de `PCA` para reducir dimensionalidad, y ver si así podemos mejorar los resultados, o al menos mantenerlos, generando modelos mucho más simples:

In [31]:
pca = PCA(n_components=10)
pca.fit(X_train)

X_train = pca.transform(X_train)
X_test = pca.transform(X_test)

print(pca.explained_variance_ratio_)

[0.1457626  0.11261286 0.09523864 0.08331368 0.07945522 0.06685725
 0.05942892 0.05029678 0.04576356 0.0397275 ]


# Selección de hiperparámetros usando K-Fold Cross Validation

Los algoritmos con los que vamos a trabajar son *SVM*, *MLP* y *AdaBoost*. Los hiperparámetros que definen estos modelos y su aprendizaje los escogemos usando *K-Fold Cross Validation*. Gracias a esta técnica podemos usar el *training set* tanto para entrenar con distintos parámetros como para validar dichos modelos. Se puede consultar más información sobre esta técnica en la [documentación oficial de sklearn](https://scikit-learn.org/stable/modules/cross_validation.html)

## Support Vector Machine

Exploramos todas las combinaciones de parámetros y aplicamos Cross Validation:

In [32]:
# Valores que fijan la penalizacion de regularizacion
C_values = [0.3, 0.5, 0.8, 0.9, 1.0, 1.05, 1.1, 1.15, 1.2, 1.3, 1.4, 1.5, 2.0, 3.0, 5.0]

# Tipos de kernel que podemos aplicar
kernel_values = ["rbf", "linear", "poly"]

# Diccionario en el que vamos a guardar los resultados de CV
results = dict()

# Para hacer 5 fold cross validation
cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=RANDOM_SEED)

# Para precomputar la mejor combinacion de parametros
best_parameters = {
    "C": None,
    "kernel": None,
}
best_acc = 0.0

# Probamos con todas las combinaciones de parametros
for C in C_values:
    for kernel in kernel_values:
        
        # Para ver el proceso de ejecucion
        print(f"Computando CV para {C}, {kernel}")
        
        # Definimos el clasificador, usando los parametros que movemos y fijando
        # el resto de parametros necesarios. 
        clf = SVC(kernel=kernel, C=C, random_state=RANDOM_SEED)
        
        # Aplicamos el algoritmo 
        scores = cross_val_score(clf, X_train, Y_train, cv=cv, scoring='accuracy')
        
        # Guardamos los resultados
        results[C, kernel, "mean"] = scores.mean()
        results[C, kernel, "std"] = scores.std()
        
        # Comprobamos si hemos mejorado los resultados
        if results[C, kernel, "mean"] > best_acc:
            best_acc = results[C, kernel, "mean"]
            best_parameters = {
                "C": C,
                "kernel": kernel
            }

Computando CV para 0.3, rbf
Computando CV para 0.3, linear
Computando CV para 0.3, poly
Computando CV para 0.5, rbf
Computando CV para 0.5, linear
Computando CV para 0.5, poly
Computando CV para 0.8, rbf
Computando CV para 0.8, linear
Computando CV para 0.8, poly
Computando CV para 0.9, rbf
Computando CV para 0.9, linear
Computando CV para 0.9, poly
Computando CV para 1.0, rbf
Computando CV para 1.0, linear
Computando CV para 1.0, poly
Computando CV para 1.05, rbf
Computando CV para 1.05, linear
Computando CV para 1.05, poly
Computando CV para 1.1, rbf
Computando CV para 1.1, linear
Computando CV para 1.1, poly
Computando CV para 1.15, rbf
Computando CV para 1.15, linear
Computando CV para 1.15, poly
Computando CV para 1.2, rbf
Computando CV para 1.2, linear
Computando CV para 1.2, poly
Computando CV para 1.3, rbf
Computando CV para 1.3, linear
Computando CV para 1.3, poly
Computando CV para 1.4, rbf
Computando CV para 1.4, linear
Computando CV para 1.4, poly
Computando CV para 1.5, rb

Mostramos todos los parámetros con sus métricas alcanzadas:

In [33]:
pprint(results)

{(0.3, 'linear', 'mean'): 0.7133802816901408,
 (0.3, 'linear', 'std'): 0.03591549295774648,
 (0.3, 'poly', 'mean'): 0.7112676056338028,
 (0.3, 'poly', 'std'): 0.036456842550830384,
 (0.3, 'rbf', 'mean'): 0.7140845070422536,
 (0.3, 'rbf', 'std'): 0.036619718309859155,
 (0.5, 'linear', 'mean'): 0.7133802816901408,
 (0.5, 'linear', 'std'): 0.03591549295774648,
 (0.5, 'poly', 'mean'): 0.7112676056338029,
 (0.5, 'poly', 'std'): 0.038572011091913105,
 (0.5, 'rbf', 'mean'): 0.7169014084507042,
 (0.5, 'rbf', 'std'): 0.03723747624338672,
 (0.8, 'linear', 'mean'): 0.7133802816901408,
 (0.8, 'linear', 'std'): 0.03591549295774648,
 (0.8, 'poly', 'mean'): 0.7042253521126761,
 (0.8, 'poly', 'std'): 0.03831400143305758,
 (0.8, 'rbf', 'mean'): 0.7232394366197183,
 (0.8, 'rbf', 'std'): 0.02817781552639798,
 (0.9, 'linear', 'mean'): 0.7133802816901408,
 (0.9, 'linear', 'std'): 0.03591549295774648,
 (0.9, 'poly', 'mean'): 0.6985915492957746,
 (0.9, 'poly', 'std'): 0.03750289283294184,
 (0.9, 'rbf', 'mean

Tomamos ahora la mejor combinación de parámetros:

In [34]:
print(best_parameters)
print(best_acc)

{'C': 0.8, 'kernel': 'rbf'}
0.7232394366197183


## MLP

Exploramos todas las combinaciones de parámetros y aplicamos Cross Validation:

In [35]:
# Valores para la regularizacion
alpha_values = [0.001, 0.01, 0.1, 0.5]

# Valores del learning rate
lr_values = [0.001, 0.01, 0.1, 0.25, 0.5]

# Diccionario en el que vamos a guardar los resultados de CV
results = dict()

# Para hacer 5 fold cross validation
cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=RANDOM_SEED)

# Para precomputar la mejor combinacion de parametros
best_parameters = {
    "lr": None,
    "alpha": None,
}
best_acc = 0.0

# Probamos con todas las combinaciones de parametros
for lr in lr_values:
    for alpha in alpha_values:
        
        # Para ver el proceso de ejecucion
        print(f"Computando CV para {lr}, {alpha}")
        
        # Definimos el clasificador, usando los parametros que movemos y fijando
        # el resto de parametros necesarios
        clf = MLPClassifier(
            activation = "relu",
            solver = "adam",
            learning_rate_init = lr,
            alpha = alpha,
            max_iter = 10,
            random_state = RANDOM_SEED
        )
        
        # Aplicamos el algoritmo 
        scores = cross_val_score(clf, X_train, Y_train, cv=cv, scoring='accuracy')
        
        # Guardamos los resultados
        results[lr, alpha, "mean"] = scores.mean()
        results[lr, alpha, "std"] = scores.std()
        
        # Comprobamos si hemos mejorado los resultados
        if results[lr, alpha, "mean"] > best_acc:
            best_acc = results[lr, alpha, "mean"]
            best_parameters = {
                "lr": lr,
                "alpha": alpha
            }

Computando CV para 0.001, 0.001
Computando CV para 0.001, 0.01
Computando CV para 0.001, 0.1
Computando CV para 0.001, 0.5
Computando CV para 0.01, 0.001
Computando CV para 0.01, 0.01
Computando CV para 0.01, 0.1
Computando CV para 0.01, 0.5
Computando CV para 0.1, 0.001
Computando CV para 0.1, 0.01
Computando CV para 0.1, 0.1
Computando CV para 0.1, 0.5
Computando CV para 0.25, 0.001
Computando CV para 0.25, 0.01
Computando CV para 0.25, 0.1
Computando CV para 0.25, 0.5
Computando CV para 0.5, 0.001
Computando CV para 0.5, 0.01
Computando CV para 0.5, 0.1
Computando CV para 0.5, 0.5


Mostramos todos los parámetros con sus métricas alcanzadas:

In [36]:
pprint(results)

{(0.001, 0.001, 'mean'): 0.7140845070422536,
 (0.001, 0.001, 'std'): 0.036619718309859155,
 (0.001, 0.01, 'mean'): 0.7140845070422536,
 (0.001, 0.01, 'std'): 0.036619718309859155,
 (0.001, 0.1, 'mean'): 0.7140845070422536,
 (0.001, 0.1, 'std'): 0.036619718309859155,
 (0.001, 0.5, 'mean'): 0.7140845070422536,
 (0.001, 0.5, 'std'): 0.036619718309859155,
 (0.01, 0.001, 'mean'): 0.7021126760563381,
 (0.01, 0.001, 'std'): 0.030379795225525346,
 (0.01, 0.01, 'mean'): 0.7021126760563381,
 (0.01, 0.01, 'std'): 0.030379795225525346,
 (0.01, 0.1, 'mean'): 0.702112676056338,
 (0.01, 0.1, 'std'): 0.029214744384072594,
 (0.01, 0.5, 'mean'): 0.7021126760563381,
 (0.01, 0.5, 'std'): 0.0252049720678497,
 (0.1, 0.001, 'mean'): 0.6887323943661972,
 (0.1, 0.001, 'std'): 0.024557177146708017,
 (0.1, 0.01, 'mean'): 0.6901408450704226,
 (0.1, 0.01, 'std'): 0.02226956098710127,
 (0.1, 0.1, 'mean'): 0.6845070422535211,
 (0.1, 0.1, 'std'): 0.01857874078629988,
 (0.1, 0.5, 'mean'): 0.7091549295774647,
 (0.1, 0.

Tomamos ahora la mejor combinación de parámetros:

In [37]:
print(best_parameters)
print(best_acc)

{'lr': 0.001, 'alpha': 0.001}
0.7140845070422536


## AdaBoost

Exploramos todas las combinaciones de parámetros y aplicamos Cross Validation:

In [38]:
# Numero de estimadores usados en el ensemble
n_values = [40, 50, 60, 75, 100, 125, 150, 175, 200, 400]

# Valores para el learning rate
lr_values = [0.4, 0.5, 0.6, 0.75, 1.0, 1.25, 1.5, 2.0]

# Diccionario en el que vamos a guardar los resultados de CV
results = dict()

# Para hacer 5 fold cross validation
cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=RANDOM_SEED)

# Para precomputar la mejor combinacion de parametros
best_parameters = {
    "lr": None,
    "n": None,
}
best_acc = 0.0

# Probamos con todas las combinaciones de parametros
for n in n_values:
    for lr in lr_values:
        
        # Para ver el proceso de ejecucion
        print(f"Computando CV para {lr}, {alpha}")
        
        # Definimos el clasificador, usando los parametros que movemos y fijando
        # el resto de parametros necesarios
        clf = AdaBoostClassifier(
            n_estimators = n,
            learning_rate = lr,
            random_state = RANDOM_SEED
        )
        
        # Aplicamos el algoritmo 
        scores = cross_val_score(clf, X_train, Y_train, cv=cv, scoring='accuracy')
        
        # Guardamos los resultados
        results[lr, n, "mean"] = scores.mean()
        results[lr, n, "std"] = scores.std()
        
        # Comprobamos si hemos mejorado los resultados
        if results[lr, n, "mean"] > best_acc:
            best_acc = results[lr, n, "mean"]
            best_parameters = {
                "lr": lr,
                "n": n
            }

Computando CV para 0.4, 0.5
Computando CV para 0.5, 0.5
Computando CV para 0.6, 0.5
Computando CV para 0.75, 0.5
Computando CV para 1.0, 0.5
Computando CV para 1.25, 0.5
Computando CV para 1.5, 0.5
Computando CV para 2.0, 0.5
Computando CV para 0.4, 0.5
Computando CV para 0.5, 0.5
Computando CV para 0.6, 0.5
Computando CV para 0.75, 0.5
Computando CV para 1.0, 0.5
Computando CV para 1.25, 0.5
Computando CV para 1.5, 0.5
Computando CV para 2.0, 0.5
Computando CV para 0.4, 0.5
Computando CV para 0.5, 0.5
Computando CV para 0.6, 0.5
Computando CV para 0.75, 0.5
Computando CV para 1.0, 0.5
Computando CV para 1.25, 0.5
Computando CV para 1.5, 0.5
Computando CV para 2.0, 0.5
Computando CV para 0.4, 0.5
Computando CV para 0.5, 0.5
Computando CV para 0.6, 0.5
Computando CV para 0.75, 0.5
Computando CV para 1.0, 0.5
Computando CV para 1.25, 0.5
Computando CV para 1.5, 0.5
Computando CV para 2.0, 0.5
Computando CV para 0.4, 0.5
Computando CV para 0.5, 0.5
Computando CV para 0.6, 0.5
Computando C

Mostramos todos los parámetros con sus métricas alcanzadas:

In [39]:
pprint(results)

{(0.4, 40, 'mean'): 0.7028169014084507,
 (0.4, 40, 'std'): 0.036156331416882924,
 (0.4, 50, 'mean'): 0.7049295774647888,
 (0.4, 50, 'std'): 0.03989298320639026,
 (0.4, 60, 'mean'): 0.6985915492957747,
 (0.4, 60, 'std'): 0.04079647424777532,
 (0.4, 75, 'mean'): 0.6894366197183099,
 (0.4, 75, 'std'): 0.02807201609576726,
 (0.4, 100, 'mean'): 0.6809859154929577,
 (0.4, 100, 'std'): 0.03348589698019362,
 (0.4, 125, 'mean'): 0.6816901408450704,
 (0.4, 125, 'std'): 0.03403674922139315,
 (0.4, 150, 'mean'): 0.6725352112676056,
 (0.4, 150, 'std'): 0.03482178961993273,
 (0.4, 175, 'mean'): 0.676056338028169,
 (0.4, 175, 'std'): 0.03805424249607408,
 (0.4, 200, 'mean'): 0.671830985915493,
 (0.4, 200, 'std'): 0.03335977262627328,
 (0.4, 400, 'mean'): 0.6598591549295774,
 (0.4, 400, 'std'): 0.03465046366237336,
 (0.5, 40, 'mean'): 0.6922535211267605,
 (0.5, 40, 'std'): 0.03392729428294621,
 (0.5, 50, 'mean'): 0.6929577464788732,
 (0.5, 50, 'std'): 0.033655783508024006,
 (0.5, 60, 'mean'): 0.692253

Tomamos ahora la mejor combinación de parámetros:

In [40]:
print(best_parameters)
print(best_acc)

{'lr': 0.4, 'n': 50}
0.7049295774647888


# Entrenamiento usando los mejores parámetros

## Función para evaluar las clasificaciones

In [41]:
def display_classification_metrics(X_train, Y_train, X_test, Y_test, model = clf, outputs_prob = False):
    """Muestra algunas metricas en clasificacion"""
    metrics = dict()
    
    # Realizamos las predicciones
    predicted_training = clf.predict(X_train)
    predicted_test = clf.predict(X_test)
    
    # Accuracy obtenido
    metrics["train_acc"] = clf.score(X_train, Y_train)
    metrics["test_acc"]= clf.score(X_test, Y_test)
    
    # Calculamos el valor de F1
    metrics["train_f1"] = f1_score(Y_train, predicted_training, average='macro')
    metrics["test_f1"] = f1_score(Y_test, predicted_test, average='macro')

    # Calculamos el area bajo la curva ROC
    if outputs_prob is False:
        metrics["train_roc_auc"] = roc_auc_score(Y_train, clf.decision_function(X_train))
        metrics["test_roc_auc"] = roc_auc_score(Y_test, clf.decision_function(X_test))
    else:
        metrics["train_roc_auc"] = roc_auc_score(Y_train, clf.predict_proba(X_train)[:, 1])
        metrics["test_roc_auc"] = roc_auc_score(Y_test, clf.predict_proba(X_test)[:, 1])
        
    # Mostramos las metricas
    print("Metricas obtenidas:")
    pprint(metrics)
    print("")
    
    # Mostramos las matrices de confusion
    print("Matriz de confusion en training:")
    matrix_train = confusion_matrix(Y_train, predicted_training)
    pprint(matrix_train)
    print("")

    print("Matriz de confusion en test:")
    matrix_test = confusion_matrix(Y_test, predicted_test)
    pprint(matrix_test)
    print("")

Una vez que, con *Cross Validation*, hemos explorado los posibles parámetros, procedemos a entrenar los modelos con los mejores parámetros encontrados usando ese procedimiento:

## Support Vector Machine

In [42]:
# Definimos el modelo
kernel = "rbf"
C = 0.8
clf = SVC(kernel=kernel, C=C, random_state=RANDOM_SEED)

# Entrenamos con el training
clf.fit(X_train, Y_train)

# Evaluamos en ambos datasets
Y_pred_training = clf.predict(X_train)
Y_pred_test = clf.predict(X_test)

print(f"Accuracy en training: {clf.score(X_train, Y_train)}")
print(f"Accuracy en test: {clf.score(X_test, Y_test)}")

Accuracy en training: 0.768033946251768
Accuracy en test: 0.715


Mostramos ahora métricas más avanzadas:

In [43]:
display_classification_metrics(X_train, Y_train, X_test, Y_test, model = clf)

Metricas obtenidas:
{'test_acc': 0.715,
 'test_f1': 0.550455459600142,
 'test_roc_auc': 0.7013095238095238,
 'train_acc': 0.768033946251768,
 'train_f1': 0.6267255588750387,
 'train_roc_auc': 0.855623188405797}

Matriz de confusion en training:
array([[489,  11],
       [153,  54]], dtype=int64)

Matriz de confusion en test:
array([[132,   8],
       [ 49,  11]], dtype=int64)



## MLP

In [44]:
# Definimos el modelo
clf = MLPClassifier(
    activation = "relu",
    solver = "adam",
    learning_rate_init = 0.001,
    alpha = 0.001,
    max_iter = 10,
    random_state = RANDOM_SEED
)

# Entrenamos con el training
clf.fit(X_train, Y_train)

# Evaluamos en ambos datasets
Y_pred_training = clf.predict(X_train)
Y_pred_test = clf.predict(X_test)

print(f"Accuracy en training: {clf.score(X_train, Y_train)}")
print(f"Accuracy en test: {clf.score(X_test, Y_test)}")

Accuracy en training: 0.7086280056577087
Accuracy en test: 0.7


Mostramos ahora métricas más avanzadas:

In [45]:
display_classification_metrics(X_train, Y_train, X_test, Y_test, model = clf, outputs_prob = True)

Metricas obtenidas:
{'test_acc': 0.7,
 'test_f1': 0.4117647058823529,
 'test_roc_auc': 0.6491666666666667,
 'train_acc': 0.7086280056577087,
 'train_f1': 0.41940139048348,
 'train_roc_auc': 0.6944927536231884}

Matriz de confusion en training:
array([[500,   0],
       [206,   1]], dtype=int64)

Matriz de confusion en test:
array([[140,   0],
       [ 60,   0]], dtype=int64)



## AdaBoost

In [46]:
# Definimos el modelo
clf = AdaBoostClassifier(
    n_estimators = 50,
    learning_rate = 0.4,
    random_state = RANDOM_SEED
)

# Entrenamos con el training
clf.fit(X_train, Y_train)

# Evaluamos en ambos datasets
Y_pred_training = clf.predict(X_train)
Y_pred_test = clf.predict(X_test)

print(f"Accuracy en training: {clf.score(X_train, Y_train)}")
print(f"Accuracy en test: {clf.score(X_test, Y_test)}")

Accuracy en training: 0.7821782178217822
Accuracy en test: 0.705


Mostramos ahora métricas más avanzadas:

In [47]:
display_classification_metrics(X_train, Y_train, X_test, Y_test, model = clf)

Metricas obtenidas:
{'test_acc': 0.705,
 'test_f1': 0.5927523727351165,
 'test_roc_auc': 0.7205357142857143,
 'train_acc': 0.7821782178217822,
 'train_f1': 0.6709918773872263,
 'train_roc_auc': 0.8249468599033816}

Matriz de confusion en training:
array([[482,  18],
       [136,  71]], dtype=int64)

Matriz de confusion en test:
array([[123,  17],
       [ 42,  18]], dtype=int64)



Por tanto, estamos viendo los mismos resultados que sin aplicar PCA. No hemos mejorado enormemente las métricas, pero estamos trabajando con modelos muchísimo más simples, y por tatno mucho más generalizables y robustos